# 혼잡도




```
공통	list_total_count	총 데이터 건수 (정상조회 시 출력됨)
공통	RESULT.CODE	요청결과 코드 (하단 메세지설명 참고)
공통	RESULT.MESSAGE	요청결과 메시지 (하단 메세지설명 참고)
1	LINK_ID	링크 아이디
2	PRCS_SPD	속도
3	PRCS_TRV_TIME	여행 시간
```



In [1]:
## 필요한 라이브러리 임포트
import xml.etree.ElementTree as ET
import requests
import pprint
import json
import pandas as pd
import numpy as np

In [3]:
### 자치구명은 직접 지도 찾아서 추가하였습니다.
### 도로링크 불러오기
df= pd.read_excel("/content/서울시 서비스링크 정보_2024년3월 기준.xlsx")
df.head()

,서비스링크,도로명,시작노드명,종료노드명,지도거리,Unnamed: 5
0,2210000500,강변북로,토평삼거리,강변북로구리시계,1997,NaN
1,1040004100,강변북로,강변북로구리시계,광진교북단,1478,광진구
2,1040002100,강변북로,광진교북단,천호대교북단,258,광진구
3,1040001500,강변북로,천호대교북단,올림픽대교북단,897,광진구
4,1040001100,강변북로,올림픽대교북단,잠실철교북단,662,광진구


In [4]:
df = df.rename(columns={'Unnamed: 5': '자치구명'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5108 entries, 0 to 5107
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   서비스링크   5108 non-null   int64 
 1   도로명     5108 non-null   object
 2   시작노드명   5108 non-null   object
 3   종료노드명   5108 non-null   object
 4   지도거리    5108 non-null   int64 
 5   자치구명    5042 non-null   object
dtypes: int64(2), object(4)
memory usage: 239.6+ KB


In [ ]:
results = []

for road in df['서비스링크']:
  API_KEY=''
  url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/xml/TrafficInfo/1/5/{road}"
  res = requests.get(url)
  contents = res.text
  root = ET.fromstring(contents)
  prcs_spd = root.find(".//row/prcs_spd").text if root.find(".//row/prcs_spd") is not None else None
  prcs_trv_time = root.find(".//row/prcs_trv_time").text if root.find(".//row/prcs_trv_time") is not None else None

  results.append({"서비스링크": road, "prcs_spd": prcs_spd, "prcs_trv_time": prcs_trv_time})


In [6]:
df1 = pd.DataFrame(results)
df1.head()

,서비스링크,prcs_spd,prcs_trv_time
0,2210000500,80,90
1,1040004100,89,60
2,1040002100,89,10
3,1040001500,78,41
4,1040001100,75,32


In [7]:
# 서비스링크를 기준으로 병합
df2 = df.merge(df1, on='서비스링크', how='left')
df2.head()

,서비스링크,도로명,시작노드명,종료노드명,지도거리,자치구명,prcs_spd,prcs_trv_time
0,2210000500,강변북로,토평삼거리,강변북로구리시계,1997,NaN,80,90
1,1040004100,강변북로,강변북로구리시계,광진교북단,1478,광진구,89,60
2,1040002100,강변북로,광진교북단,천호대교북단,258,광진구,89,10
3,1040001500,강변북로,천호대교북단,올림픽대교북단,897,광진구,78,41
4,1040001100,강변북로,올림픽대교북단,잠실철교북단,662,광진구,75,32


In [8]:
df2 = df.merge(df1, on='서비스링크', how='left')
df2 = df2.dropna()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5007 entries, 1 to 5107
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   서비스링크          5007 non-null   int64 
 1   도로명            5007 non-null   object
 2   시작노드명          5007 non-null   object
 3   종료노드명          5007 non-null   object
 4   지도거리           5007 non-null   int64 
 5   자치구명           5007 non-null   object
 6   prcs_spd       5007 non-null   object
 7   prcs_trv_time  5007 non-null   object
dtypes: int64(2), object(6)
memory usage: 352.1+ KB


In [9]:
df2.to_excel("혼잡도_18시.xlsx")

In [10]:
df2

,서비스링크,도로명,시작노드명,종료노드명,지도거리,자치구명,prcs_spd,prcs_trv_time
1,1040004100,강변북로,강변북로구리시계,광진교북단,1478,광진구,89,60
2,1040002100,강변북로,광진교북단,천호대교북단,258,광진구,89,10
3,1040001500,강변북로,천호대교북단,올림픽대교북단,897,광진구,78,41
4,1040001100,강변북로,올림픽대교북단,잠실철교북단,662,광진구,75,32
5,1040000500,강변북로,잠실철교북단,잠실대교북단,869,광진구,67,47
...,...,...,...,...,...,...,...,...
5103,1000034400,새문안로3길,경복궁역,적선현대빌딩,67,종로구,14.9,16
5104,1000032500,새문안로3길,적선현대빌딩,서울지방경찰청,137,종로구,2,247
5105,1000031200,새문안로3길,서울지방경찰청,용비어천가,109,종로구,26.3,15
5106,1000028200,새문안로3길,용비어천가,세종아파트앞,161,종로구,16.6,35


In [11]:
len(df2['자치구명'].unique())

28

In [12]:
df2['자치구명'].unique()

## 깅서구 -> '강서구'
## 영등포 -> '영등포구'
## 남대문구 -> '중구'
seoul = {
    '깅서구': '강서구',
    '영등포': '영등포구',
    '남대문구': '중구'  # 서울에 '남대문구'는 없으므로 '중구'로 변경
}

# 값 수정
df2['자치구명'] = df2['자치구명'].replace(seoul)
len(df2['자치구명'].unique())

25

In [13]:
df2['prcs_spd'] = pd.to_numeric(df2['prcs_spd'], errors='coerce')
df2['prcs_trv_time'] = pd.to_numeric(df2['prcs_trv_time'], errors='coerce')

In [14]:
# 자치구별 prcs_spd, prcs_trv_time 평균
df_final = df2.groupby('자치구명')[['prcs_spd', 'prcs_trv_time']].mean().reset_index()
df_final

,자치구명,prcs_spd,prcs_trv_time
0,강남구,20.475309,142.095679
1,강동구,21.266556,105.523179
2,강북구,19.040496,122.471074
3,강서구,21.988426,130.504630
4,관악구,26.572000,125.940000
5,광진구,22.045390,125.815603
6,구로구,23.341237,97.546392
7,금천구,22.250420,125.184874
8,노원구,21.715417,99.720833
9,도봉구,21.465476,93.595238


In [15]:
# 혼잡도 점수 계산 : 점수 기준으로 3등분하여 분류
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_final[['이동속도', '이동시간']] = scaler.fit_transform(df_final[['prcs_spd', 'prcs_trv_time']])

df_final['혼잡도 점수'] = (1 - df_final['이동속도']) + df_final['이동시간']
df_final['도로 혼잡도'] = pd.qcut(df_final['혼잡도 점수'], q=3, labels=['원활', '보통', '혼잡'])


In [16]:
df_final

,자치구명,prcs_spd,prcs_trv_time,이동속도,이동시간,혼잡도 점수,도로 혼잡도
0,강남구,20.475309,142.095679,0.266286,1.000000,1.733714,혼잡
1,강동구,21.266556,105.523179,0.361510,0.394114,1.032605,보통
2,강북구,19.040496,122.471074,0.093612,0.674885,1.581273,혼잡
3,강서구,21.988426,130.504630,0.448384,0.807975,1.359591,혼잡
4,관악구,26.572000,125.940000,1.000000,0.732354,0.732354,원활
5,광진구,22.045390,125.815603,0.455239,0.730293,1.275053,보통
6,구로구,23.341237,97.546392,0.611190,0.261965,0.650776,원활
7,금천구,22.250420,125.184874,0.479914,0.719844,1.239930,보통
8,노원구,21.715417,99.720833,0.415528,0.297989,0.882460,원활
9,도봉구,21.465476,93.595238,0.385449,0.196508,0.811059,원활


In [17]:
df_final['도로 혼잡도'].value_counts()

,count
도로 혼잡도,
원활,9
보통,8
혼잡,8


In [18]:
df_final.to_excel("자치구별_도로혼잡도_18시.xlsx")

In [ ]:
df18 = pd.read_excel("/content/자치구별_도로혼잡도_18시.xlsx")
df21 = pd.read_excel